In [7]:
def rules(clf, features, labels, node_index=0):
    """Structure of rules in a fit decision tree classifier

    Parameters
    ----------
    clf : DecisionTreeClassifier
        A tree that has already been fit.

    features, labels : lists of str
        The names of the features and labels, respectively.

    """
    node = {}
    if clf.tree_.children_left[node_index] == -1:  # indicates leaf
        count_labels = zip(clf.tree_.value[node_index, 0], labels)
        node['name'] = ', '.join(('{} of {}'.format(int(count), label)
                                  for count, label in count_labels))
    else:
        feature = features[clf.tree_.feature[node_index]]
        threshold = clf.tree_.threshold[node_index]
        node['name'] = '{} > {}'.format(feature, threshold)
        left_index = clf.tree_.children_left[node_index]
        right_index = clf.tree_.children_right[node_index]
        node['children'] = [rules(clf, features, labels, right_index),
                            rules(clf, features, labels, left_index)]
    return node

In [8]:
from sklearn.datasets import load_iris, load_boston, load_breast_cancer

from sklearn.tree import DecisionTreeClassifier

data = load_breast_cancer()

clf = DecisionTreeClassifier(max_depth=3)
clf.fit(data.data, data.target)

rules(clf, data.feature_names, data.target_names)

{'name': 'worst radius > 16.795000076293945',
 'children': [{'name': 'worst texture > 19.90999984741211',
   'children': [{'name': 'worst smoothness > 0.0879799984395504',
     'children': [{'name': '171 of malignant, 1 of benign'},
      {'name': '0 of malignant, 1 of benign'}]},
    {'name': 'concavity error > 0.03371500037610531',
     'children': [{'name': '8 of malignant, 0 of benign'},
      {'name': '0 of malignant, 9 of benign'}]}]},
  {'name': 'worst concave points > 0.13580000400543213',
   'children': [{'name': 'worst texture > 25.670000076293945',
     'children': [{'name': '24 of malignant, 3 of benign'},
      {'name': '4 of malignant, 15 of benign'}]},
    {'name': 'radius error > 1.047549992799759',
     'children': [{'name': '1 of malignant, 0 of benign'},
      {'name': '4 of malignant, 328 of benign'}]}]}]}

In [9]:
import json

r = rules(clf, data.feature_names, data.target_names)
with open('breast_cancer.json', 'w') as f:
    f.write(json.dumps(r))